In [1]:
#importing the libraries
import requests 
from bs4 import BeautifulSoup as BS
import re
import pandas as pd
import numpy as np

#getting all webpages of each county in the Bay area

In [2]:

 x = ['https://www.redfin.com/city/17151/CA/San-Francisco' ,
      'https://www.redfin.com/county/303/CA/Alameda-County'
#      'https://www.redfin.com/county/323/CA/Marin-County',
#      'https://www.redfin.com/county/330/CA/Napa-County',
#      'https://www.redfin.com/county/309/CA/Contra-Costa-County', 
#      'https://www.redfin.com/county/343/CA/San-Mateo-County',
#      'https://www.redfin.com/county/345/CA/Santa-Clara-County',
#      'https://www.redfin.com/county/350/CA/Solano-County',
#      'https://www.redfin.com/county/351/CA/Sonoma-County'
    ]

# getting the multiple page links for each of above county from the website for scraping
county_links =[]
for c_l in x:
    c_text = requests.get(c_l,headers={'User-Agent': 'Mozilla/5.0'}).text
    c_soup = BS(c_text,'html5lib') 
    page_number=int(c_soup.find('span',attrs={'class':'pageText'}).text.split('of ')[1])
    county_links.append(c_l)
    if page_number>1:
        for num in range(2,page_number+1):
            str_num=str(num)
            new_c_l = c_l+'/page-'+str_num
            county_links.append(new_c_l)

county_links

['https://www.redfin.com/city/17151/CA/San-Francisco',
 'https://www.redfin.com/city/17151/CA/San-Francisco/page-2',
 'https://www.redfin.com/city/17151/CA/San-Francisco/page-3',
 'https://www.redfin.com/city/17151/CA/San-Francisco/page-4',
 'https://www.redfin.com/city/17151/CA/San-Francisco/page-5',
 'https://www.redfin.com/city/17151/CA/San-Francisco/page-6',
 'https://www.redfin.com/city/17151/CA/San-Francisco/page-7',
 'https://www.redfin.com/city/17151/CA/San-Francisco/page-8',
 'https://www.redfin.com/city/17151/CA/San-Francisco/page-9',
 'https://www.redfin.com/county/303/CA/Alameda-County',
 'https://www.redfin.com/county/303/CA/Alameda-County/page-2',
 'https://www.redfin.com/county/303/CA/Alameda-County/page-3',
 'https://www.redfin.com/county/303/CA/Alameda-County/page-4',
 'https://www.redfin.com/county/303/CA/Alameda-County/page-5',
 'https://www.redfin.com/county/303/CA/Alameda-County/page-6',
 'https://www.redfin.com/county/303/CA/Alameda-County/page-7',
 'https://www.r

In [3]:
#Creating a dictionary with key - county , value - List of properties
#extracting all links in a webpage using a href tags and using regex to identify the links for properties

Property_set={}

for url in county_links:
    county_url = url
    county_name_list = re.findall('CA/([^/]+)',county_url)
    text = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}).text
    soup = BS(text,'html5lib')
    all_links = [a['href'] for a in soup('a') if a.has_attr('href')] #extracting all links on the webpage
    regex = r"^/CA" #extracting only links for properties listed
    one_county_prop_url = [link for link in all_links if re.match(regex,link)]
    for a in range(0, len(one_county_prop_url)):
        one_county_prop_url[a]='https://www.redfin.com'+one_county_prop_url[a]
    final_url =[]
    for l in one_county_prop_url:
        if l not in final_url:
            final_url.append(l)
    for x in county_name_list:
        if x in Property_set:
            Property_set[x]=Property_set[x]+final_url
        else:
            Property_set[x]=final_url
        
Property_set

{'San-Francisco': ['https://www.redfin.com/CA/San-Francisco/1830-Fillmore-St-94115/unit-311/home/1030296',
  'https://www.redfin.com/CA/San-Francisco/2152-16th-Ave-94116/home/1022996',
  'https://www.redfin.com/CA/San-Francisco/2170-Jackson-St-94115/unit-1/home/12395510',
  'https://www.redfin.com/CA/San-Francisco/1631-La-Salle-Ave-94124/home/748687',
  'https://www.redfin.com/CA/San-Francisco/350-Day-St-94131/home/1744272',
  'https://www.redfin.com/CA/San-Francisco/1824-Jackson-St-94109/home/2020852',
  'https://www.redfin.com/CA/San-Francisco/301-Crescent-Ct-94134/unit-3101/home/12331178',
  'https://www.redfin.com/CA/San-Francisco/2900-Scott-St-94123/home/1707827',
  'https://www.redfin.com/CA/San-Francisco/1050-N-Point-St-94109/unit-302/home/12400200',
  'https://www.redfin.com/CA/San-Francisco/2171-Mission-St-94110/unit-302/home/1658102',
  'https://www.redfin.com/CA/San-Francisco/1000-N-Point-St-94109/unit-1505/home/12395450',
  'https://www.redfin.com/CA/San-Francisco/1483-Sutt

In [4]:
#creating empty lists for the attributes and extracting property information from above links using tags like div, span, etc

Street= []
City = []
State = []
Zipcode = []
Price = []
Beds = []
Bath = []
Sqft = []
Property_Type = []
Year_Built = []
Status = []
Walk_Score = []
Bike_Score = []
Acre=[]
count=0

for county in Property_set:
    for prop in Property_set[county]:
        print(count)
        count=count+1
        al_url = prop
        al_text = requests.get(al_url,headers={'User-Agent': 'Mozilla/5.0'}).text
        al_soup = BS(al_text,'html5lib')

#using try & except blocks to identify errors and handle them

        try:
            street = al_soup.find('div',attrs={'class':'street-address'}).text.split(',')[0]
        except:
            street=np.nan
            print(al_url)
            #print('street')
        Street.append(street)
        
        try:
            city = al_soup.find('div',attrs={'class':'dp-subtext'}).text.split(', ')[0]
        except:
            city=np.nan
            print(al_url)
            #print('city')
        City.append(city)
        
        try:
            state = al_soup.find('div',attrs={'class':'dp-subtext'}).text.split(', ')[1].split(' ')[0]
        except:
            state=np.nan
            print(al_url)
            #print('state')
        State.append(state)
        
        try:
            zipcode = al_soup.find('div',attrs={'class':'dp-subtext'}).text.split(', ')[-1].split(' ')[-1]
            
        except:
            zipcode=np.nan
            print(al_url)
            #print('zipcode')
        Zipcode.append(zipcode)    
        
        try:
            price = al_soup.find('div',attrs={'class':'statsValue'}).text.split('$')[1]
        except:
            price=np.nan
            print(al_url)
            #print('price')
        Price.append(price)
        
        try:
            bed_str = al_soup.find('div',attrs={'data-rf-test-id':'abp-beds'}).text.split('B')[0]
            beds = int(bed_str)
        except:
            beds=np.nan
            print(al_url)
            #print('beds')
        Beds.append(beds)
        
        try:
            bath_str =al_soup.find('div',attrs ={'data-rf-test-id':'abp-baths'}).text.split('B')[0]
            bath = float(bath_str)
        except:
            bath=np.nan
            print(al_url)
            #print('bath')
        Bath.append(bath)
        
        try:
            sqft_str =al_soup.find('div',attrs ={'data-rf-test-id':'abp-sqFt'}).text.split('S')[0].replace(',','')
            sqft = float(sqft_str)
        except:
            sqft=np.nan
            print(al_url)
            #print('sqft')
        Sqft.append(sqft)
        
        try:
            acre_str =al_soup.find('div',attrs ={'data-rf-test-id':'abp-sqFt'}).text.split('A')[0]
            acre = float(acre_str)
        except:
            acre=np.nan
            print(al_url)
            #print('acre')
        Acre.append(acre)
        
        
        try:
            div_prop = al_soup.findAll('div',attrs={'class':'keyDetail font-weight-roman font-size-base'})
            property_type =''
            for tag in div_prop:
                if(tag.span.text=='Property Type'):
                    property_type = property_type+tag.find('span',attrs={'class':'content text-right'}).text
        except:
            property_type = np.nan
            print(al_url)
            #print('prop_type')
        Property_Type.append(property_type)
        
        try:
            div_year_built = al_soup.findAll('div',attrs={'class':'keyDetail font-weight-roman font-size-base'})
            year_built =''
            for tag in div_year_built:
                if(tag.span.text=='Year Built'):
                    year_built = int(year_built+tag.find('span',attrs={'class':'content text-right'}).text)
        except:
            year_built=np.nan
            print(al_url)
            #print('year')
        Year_Built.append(year_built)
        
        try:
            div_status = al_soup.findAll('div',attrs={'class':'keyDetail font-weight-roman font-size-base'})
            status =''
            for tag in div_status:
                if(tag.span.text=='Status'):
                    status = tag.find('span',attrs={'class':'DefinitionFlyoutLink inline-block underline clickable'})
                    if status == None:
                        status = tag.find('span',attrs={'class':'content text-right'})
        except:
            status=np.nan
            print(al_url)
            #print('status')
        Status.append(status.text)

        try:
            walk_score = int(al_soup.find('div',attrs={'class':'transport-icon-and-percentage walkscore'}).text.split(' /')[0])
        except:
            walk_score=np.nan
            print(al_url)
            #print('walk')
        Walk_Score.append(walk_score)
        
        try:
            bike_score = int(al_soup.find('div',attrs={'class':'transport-icon-and-percentage bikescore'}).text.split(' /')[0])
        except:
            bike_score=np.nan
            print(al_url)
            #print('bike')
        Bike_Score.append(bike_score)
       


0
https://www.redfin.com/CA/San-Francisco/1830-Fillmore-St-94115/unit-311/home/1030296
https://www.redfin.com/CA/San-Francisco/1830-Fillmore-St-94115/unit-311/home/1030296
https://www.redfin.com/CA/San-Francisco/1830-Fillmore-St-94115/unit-311/home/1030296
1
https://www.redfin.com/CA/San-Francisco/2152-16th-Ave-94116/home/1022996
https://www.redfin.com/CA/San-Francisco/2152-16th-Ave-94116/home/1022996
https://www.redfin.com/CA/San-Francisco/2152-16th-Ave-94116/home/1022996
2
https://www.redfin.com/CA/San-Francisco/2170-Jackson-St-94115/unit-1/home/12395510
3
https://www.redfin.com/CA/San-Francisco/1631-La-Salle-Ave-94124/home/748687
https://www.redfin.com/CA/San-Francisco/1631-La-Salle-Ave-94124/home/748687
https://www.redfin.com/CA/San-Francisco/1631-La-Salle-Ave-94124/home/748687
4
https://www.redfin.com/CA/San-Francisco/350-Day-St-94131/home/1744272
https://www.redfin.com/CA/San-Francisco/350-Day-St-94131/home/1744272
5
https://www.redfin.com/CA/San-Francisco/1824-Jackson-St-94109/h

https://www.redfin.com/CA/San-Francisco/601-Van-Ness-Ave-94102/unit-311/home/1288564
53
https://www.redfin.com/CA/San-Francisco/1552-Chestnut-St-94123/home/1858706
https://www.redfin.com/CA/San-Francisco/1552-Chestnut-St-94123/home/1858706
54
https://www.redfin.com/CA/San-Francisco/300-Ivy-St-94102/unit-303/home/69054841
https://www.redfin.com/CA/San-Francisco/300-Ivy-St-94102/unit-303/home/69054841
https://www.redfin.com/CA/San-Francisco/300-Ivy-St-94102/unit-303/home/69054841
55
https://www.redfin.com/CA/San-Francisco/964-Potrero-Ave-94110/home/17306338
https://www.redfin.com/CA/San-Francisco/964-Potrero-Ave-94110/home/17306338
56
https://www.redfin.com/CA/San-Francisco/1001-Pine-St-94109/unit-509/home/1586585
57
https://www.redfin.com/CA/San-Francisco/1255-19th-Ave-94122/home/1531342
58
https://www.redfin.com/CA/San-Francisco/3131-Pierce-St-94123/unit-403/home/174645267
59
https://www.redfin.com/CA/San-Francisco/208-17th-Ave-94121/home/110051527
https://www.redfin.com/CA/San-Francis

https://www.redfin.com/CA/San-Francisco/1506-41st-Ave-94122/home/1403454
110
https://www.redfin.com/CA/San-Francisco/1639-46th-Ave-94122/home/636387
https://www.redfin.com/CA/San-Francisco/1639-46th-Ave-94122/home/636387
https://www.redfin.com/CA/San-Francisco/1639-46th-Ave-94122/home/636387
111
https://www.redfin.com/CA/San-Francisco/187-Ellsworth-St-94110/home/40629238
https://www.redfin.com/CA/San-Francisco/187-Ellsworth-St-94110/home/40629238
https://www.redfin.com/CA/San-Francisco/187-Ellsworth-St-94110/home/40629238
112
https://www.redfin.com/CA/San-Francisco/1050-N-Point-St-94109/unit-206/home/12400769
113
https://www.redfin.com/CA/San-Francisco/658-Hayes-St-94102/home/852358
https://www.redfin.com/CA/San-Francisco/658-Hayes-St-94102/home/852358
https://www.redfin.com/CA/San-Francisco/658-Hayes-St-94102/home/852358
114
https://www.redfin.com/CA/San-Francisco/822-28th-Ave-94121/home/623485
https://www.redfin.com/CA/San-Francisco/822-28th-Ave-94121/home/623485
https://www.redfin.c

https://www.redfin.com/CA/San-Francisco/392-Corbett-Ave-94114/home/181892817
https://www.redfin.com/CA/San-Francisco/392-Corbett-Ave-94114/home/181892817
https://www.redfin.com/CA/San-Francisco/392-Corbett-Ave-94114/home/181892817
156
https://www.redfin.com/CA/San-Francisco/472-Anderson-St-94110/home/1591526
https://www.redfin.com/CA/San-Francisco/472-Anderson-St-94110/home/1591526
https://www.redfin.com/CA/San-Francisco/472-Anderson-St-94110/home/1591526
157
https://www.redfin.com/CA/San-Francisco/390-Corbett-Ave-94114/home/674375
158
https://www.redfin.com/CA/San-Francisco/731-Moultrie-St-94110/home/764866
https://www.redfin.com/CA/San-Francisco/731-Moultrie-St-94110/home/764866
159
https://www.redfin.com/CA/San-Francisco/3953-Washington-St-94118/home/1492099
160
https://www.redfin.com/CA/San-Francisco/24-Cuvier-St-94112/home/1142171
161
https://www.redfin.com/CA/San-Francisco/2371-34th-Ave-94116/home/656672
https://www.redfin.com/CA/San-Francisco/2371-34th-Ave-94116/home/656672
http

https://www.redfin.com/CA/San-Francisco/Undisclosed-address-94114/home/1458392
https://www.redfin.com/CA/San-Francisco/Undisclosed-address-94114/home/1458392
https://www.redfin.com/CA/San-Francisco/Undisclosed-address-94114/home/1458392
201
https://www.redfin.com/CA/San-Francisco/1969-California-St-94109/home/1434591
202
https://www.redfin.com/CA/San-Francisco/2-Mint-Plz-94103/unit-502/home/12397588
https://www.redfin.com/CA/San-Francisco/2-Mint-Plz-94103/unit-502/home/12397588
https://www.redfin.com/CA/San-Francisco/2-Mint-Plz-94103/unit-502/home/12397588
203
https://www.redfin.com/CA/San-Francisco/1608-Folsom-St-94103/home/1694641
https://www.redfin.com/CA/San-Francisco/1608-Folsom-St-94103/home/1694641
https://www.redfin.com/CA/San-Francisco/1608-Folsom-St-94103/home/1694641
204
https://www.redfin.com/CA/San-Francisco/381-Park-St-94110/home/1406781
https://www.redfin.com/CA/San-Francisco/381-Park-St-94110/home/1406781
https://www.redfin.com/CA/San-Francisco/381-Park-St-94110/home/14

...

https://www.redfin.com/CA/Oakland/177-19th-St-94612/unit-11C/home/17183152
https://www.redfin.com/CA/Oakland/177-19th-St-94612/unit-11C/home/17183152
https://www.redfin.com/CA/Oakland/177-19th-St-94612/unit-11C/home/17183152
458
https://www.redfin.com/CA/San-Leandro/157-Santa-Teresa-94579/unit-157/home/126928379
https://www.redfin.com/CA/San-Leandro/157-Santa-Teresa-94579/unit-157/home/126928379
https://www.redfin.com/CA/San-Leandro/157-Santa-Teresa-94579/unit-157/home/126928379
459
https://www.redfin.com/CA/Dublin/7720-Donohue-Dr-94568/home/1166046
460
https://www.redfin.com/CA/Fremont/5322-Tallman-Ct-94536/home/1215795
461
https://www.redfin.com/CA/Oakland/2730-Seminary-Ave-94605/home/567735
462
https://www.redfin.com/CA/Livermore/5648-Dresslar-Cir-94550/home/1709880
463
https://www.redfin.com/CA/Union-City/34717-Alvarado-Niles-Rd-94587/unit-3/home/842741
464
https://www.redfin.com/CA/Oakland/4908-Fairfax-Ave-94601/home/1356642
465
https://www.redfin.com/CA/San-Leandro/13828-Velarde-

https://www.redfin.com/CA/Fremont/45158-Tom-Blalock-St-94539/unit-204/home/179702422
https://www.redfin.com/CA/Fremont/45158-Tom-Blalock-St-94539/unit-204/home/179702422
https://www.redfin.com/CA/Fremont/45158-Tom-Blalock-St-94539/unit-204/home/179702422
510
https://www.redfin.com/CA/San-Lorenzo/15725-Wagner-St-94580/home/1824762
511
https://www.redfin.com/CA/Dublin/3307-Longmeadow-Pl-94568/home/1879486
512
https://www.redfin.com/CA/Oakland/767-Calmar-Ave-94610/home/527249
513
https://www.redfin.com/CA/Fremont/1380-Striper-Cmn-94536/home/1069952
https://www.redfin.com/CA/Fremont/1380-Striper-Cmn-94536/home/1069952
https://www.redfin.com/CA/Fremont/1380-Striper-Cmn-94536/home/1069952
514
https://www.redfin.com/CA/Alameda/1036-Melrose-Ave-94502/home/1280663
515
https://www.redfin.com/CA/Fremont/47850-Masters-Ct-94539/home/1280036
516
https://www.redfin.com/CA/Berkeley/2439-Roosevelt-Ave-94703/home/668240
https://www.redfin.com/CA/Berkeley/2439-Roosevelt-Ave-94703/home/668240
https://www.

https://www.redfin.com/CA/Fremont/3207-Kemper-Rd-94536/home/587542
567
https://www.redfin.com/CA/OAKLAND/200-LAKESIDE-DR-94612/unit-503/home/530546
568
https://www.redfin.com/CA/Oakland/229-International-Blvd-94606/home/95474959
https://www.redfin.com/CA/Oakland/229-International-Blvd-94606/home/95474959
https://www.redfin.com/CA/Oakland/229-International-Blvd-94606/home/95474959
https://www.redfin.com/CA/Oakland/229-International-Blvd-94606/home/95474959
569
https://www.redfin.com/CA/Livermore/5291-Lilac-Ave-94551/home/1895999
570
https://www.redfin.com/CA/Oakland/1044-86th-Ave-94621/home/582314
571
https://www.redfin.com/CA/Hayward/30021-Mountain-View-Dr-94544/home/143116794
https://www.redfin.com/CA/Hayward/30021-Mountain-View-Dr-94544/home/143116794
https://www.redfin.com/CA/Hayward/30021-Mountain-View-Dr-94544/home/143116794
572
https://www.redfin.com/CA/Livermore/5330-Sandra-Way-94550/home/1767427
573
https://www.redfin.com/CA/Berkeley/2635-Mathews-St-94702/home/1291537
574
https

https://www.redfin.com/CA/Union-City/2529-Trailside-Way-94587/home/2039020
629
https://www.redfin.com/CA/Oakland/5431-Holland-St-94601/home/554872
630
https://www.redfin.com/CA/Berkeley/20-Brookside-Ave-94705/home/1818729
631
https://www.redfin.com/CA/Castro-Valley/4147-Ravenwood-Pl-94546/home/828132
632
https://www.redfin.com/CA/Berkeley/1635-Scenic-Ave-94709/unit-3/home/2006525
633
https://www.redfin.com/CA/Oakland/620-Alcatraz-Ave-94609/unit-A/home/17188238
634
https://www.redfin.com/CA/Oakland/631-Wesley-Ave-94610/home/535725
635
https://www.redfin.com/CA/Livermore/2745-Kellogg-Loop-94550/home/2110891
https://www.redfin.com/CA/Livermore/2745-Kellogg-Loop-94550/home/2110891
https://www.redfin.com/CA/Livermore/2745-Kellogg-Loop-94550/home/2110891
636
https://www.redfin.com/CA/San-Leandro/14301-Santiago-Rd-94577/home/799473
637
https://www.redfin.com/CA/Castro-Valley/18426-Carlwyn-Dr-94546/home/1422565
638
https://www.redfin.com/CA/Hayward/362-Jerilynn-Ln-94541/home/1812909
https://ww

https://www.redfin.com/CA/Hayward/26517-Sunvale-Ct-94544/unit-3/home/1002786
https://www.redfin.com/CA/Hayward/26517-Sunvale-Ct-94544/unit-3/home/1002786
https://www.redfin.com/CA/Hayward/26517-Sunvale-Ct-94544/unit-3/home/1002786
685
https://www.redfin.com/CA/Alameda/185-Parfait-Ln-94502/home/1966912
686
https://www.redfin.com/CA/Oakland/10530-Stella-St-94605/home/17188121
687
https://www.redfin.com/CA/Oakland/3110-Park-Blvd-94610/home/22812710
https://www.redfin.com/CA/Oakland/3110-Park-Blvd-94610/home/22812710
https://www.redfin.com/CA/Oakland/3110-Park-Blvd-94610/home/22812710
https://www.redfin.com/CA/Oakland/3110-Park-Blvd-94610/home/22812710
https://www.redfin.com/CA/Oakland/3110-Park-Blvd-94610/home/22812710
688
https://www.redfin.com/CA/Oakland/4830-Ygnacio-Ave-94601/home/1456706
689
https://www.redfin.com/CA/Oakland/4424-Camden-St-94619/home/2036528
https://www.redfin.com/CA/Oakland/4424-Camden-St-94619/home/2036528
https://www.redfin.com/CA/Oakland/4424-Camden-St-94619/home/

In [5]:
County=[]
for key in Property_set:
    for value in Property_set[key]:
       County.append(key)
np.unique(County)

array(['Alameda-County', 'San-Francisco'], dtype='<U14')

In [6]:
#dataframe of all above 

In [7]:
df=pd.DataFrame(data={'Street':Street,
                      'City':City,
                      'County':County,
                      'State':State,
                      'Zipcode':Zipcode,
                      'Price':Price,
                      'WalkScore':Walk_Score,
                      'Bike_Score':Bike_Score,
                      'Beds':Beds,
                      'Bath':Bath,
                      'Sqft':Sqft,
                      'Acre':Acre,
                      'Property_Type':Property_Type,
                      'Year_Built':Year_Built,
                      'Status':Status})

In [8]:
df     #printing to check current data

,Street,City,County,State,Zipcode,Price,WalkScore,Bike_Score,Beds,Bath,Sqft,Acre,Property_Type,Year_Built,Status
0,1830 Fillmore St #311,SAN FRANCISCO,San-Francisco,CA,94115,"1,098,000",NaN,NaN,2.0,2.0,1107.0,NaN,Condo,1984,Active
1,2152 16th Ave,San Francisco,San-Francisco,CA,94116,"1,298,000",NaN,NaN,3.0,2.0,1325.0,NaN,Single Family Residential,1938,Coming Soon
2,2170 Jackson St #1,San Francisco,San-Francisco,CA,94115,"4,525,000",97.0,64.0,3.0,3.5,3428.0,NaN,Co-op,1925,Active
3,1631 La Salle Ave,San Francisco,San-Francisco,CA,94124,"1,495,000",NaN,NaN,4.0,4.0,2430.0,NaN,Single Family Residential,1940,Active
4,350 Day St,San Francisco,San-Francisco,CA,94131,"1,995,000",91.0,65.0,2.0,1.0,NaN,NaN,Single Family Residential,1900,Active
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
705,2214 Browning St,Berkeley,Alameda-County,CA,94702,"1,249,000",NaN,NaN,3.0,2.0,1560.0,NaN,Single Family Residential,1918,Active
706,1510 E St #22,Hayward,Alameda-County,CA,94541,"498,000",54.0,26.0,2.0,2.0,1120.0,NaN,Condo,1974,Active
707,1625 San Lorenzo Ave,Berkeley,Alameda-County,CA,94707,"1,199,000",93.0,78.0,2.0,2.0,1153.0,NaN,Single Family Residential,1925,Active
708,6689 Thorndale Dr,Oakland,Alameda-County,CA,94611,"249,000",2.0,13.0,NaN,NaN,10300.0,NaN,Vacant Land,,Active


In [9]:
df.isna().sum()

Street             0
City               0
County             0
State              0
Zipcode            0
Price              0
WalkScore        333
Bike_Score       334
Beds              34
Bath              28
Sqft              30
Acre             708
Property_Type      0
Year_Built         0
Status             0
dtype: int64

### Data cleaning

In [10]:
(df['Beds'].isna() & df['Bath'].isna()).sum()

25

In [11]:
# When both beds and Bath have null values it indicates that the property is empty land and there is no 
# built house. We will drop these rows
df = df.dropna(subset=['Beds','Bath'],how='all')

In [12]:
#Street, City, State, Zipcode, Price have small number of null values. We will remove such rows since that will affect
#our analysis and insights
df=df.dropna(subset=['Street','City','State','Zipcode','Price'])

In [13]:
# Replacing walk score and bike score null values with score of 0. We do not need to remove entire row
# based on these scores as null value indicates score of 0.
df['WalkScore'].fillna(0,inplace=True)
df['Bike_Score'].fillna(0,inplace=True)

In [14]:
# We have removed properties where both beds and baths are 0
# To deal with remaining null values in Individual columns of Beds & Baths, we will replace it with 0 since 
# that means there is either no bedroom or not bathroom in that property

df['Beds'].fillna(0,inplace=True)
df['Bath'].fillna(0,inplace=True)

In [15]:
df.isna().sum()

Street             0
City               0
County             0
State              0
Zipcode            0
Price              0
WalkScore          0
Bike_Score         0
Beds               0
Bath               0
Sqft              25
Acre             685
Property_Type      0
Year_Built         0
Status             0
dtype: int64

In [16]:
#dropping the rows where sqft is not provided(null) since we want to analyze sqft
df=df.dropna(subset=['Sqft'])

In [17]:
#we can remove the acre column. It was only used to identify the values where we do not have sqft
df=df.drop(columns=['Acre'])
df=df.reset_index()
df=df.drop(columns=['index'])
df

,Street,City,County,State,Zipcode,Price,WalkScore,Bike_Score,Beds,Bath,Sqft,Property_Type,Year_Built,Status
0,1830 Fillmore St #311,SAN FRANCISCO,San-Francisco,CA,94115,"1,098,000",0.0,0.0,2.0,2.0,1107.0,Condo,1984,Active
1,2152 16th Ave,San Francisco,San-Francisco,CA,94116,"1,298,000",0.0,0.0,3.0,2.0,1325.0,Single Family Residential,1938,Coming Soon
2,2170 Jackson St #1,San Francisco,San-Francisco,CA,94115,"4,525,000",97.0,64.0,3.0,3.5,3428.0,Co-op,1925,Active
3,1631 La Salle Ave,San Francisco,San-Francisco,CA,94124,"1,495,000",0.0,0.0,4.0,4.0,2430.0,Single Family Residential,1940,Active
4,1824 Jackson St,San Francisco,San-Francisco,CA,94109,"2,300,000",0.0,0.0,3.0,2.5,2854.0,Condo,1917,Active
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,36739 Hafner St,Newark,Alameda-County,CA,94560,"1,190,000",80.0,50.0,4.0,2.0,1216.0,Single Family Residential,1954,Active
656,2214 Browning St,Berkeley,Alameda-County,CA,94702,"1,249,000",0.0,0.0,3.0,2.0,1560.0,Single Family Residential,1918,Active
657,1510 E St #22,Hayward,Alameda-County,CA,94541,"498,000",54.0,26.0,2.0,2.0,1120.0,Condo,1974,Active
658,1625 San Lorenzo Ave,Berkeley,Alameda-County,CA,94707,"1,199,000",93.0,78.0,2.0,2.0,1153.0,Single Family Residential,1925,Active


In [18]:
# clearing +sign & special characters
for i in range(0,len(df)):
    if(df['Price'][i][-1]=='+'):
        df['Price'][i]=df['Price'][i][0:-1]  
    else:
        df['Price'][i]=df['Price'][i]
        

/var/folders/2f/l2xlm54j57qdd4g5l_gz2jl00000gn/T/ipykernel_3807/2055660007.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Price'][i]=df['Price'][i]


In [19]:
#converting data type to relevant types

df['Price'] = df['Price'].str.replace(',','')
df['Price'] = df['Price'].astype(float)
df.loc[df['Year_Built'] == '', 'Year_Built'] = 0
df['Year_Built'] = df['Year_Built'].astype(int)
df.loc[df['Year_Built'] == 0, 'Year_Built'] = np.nan

In [20]:
df.dtypes

Street            object
City              object
County            object
State             object
Zipcode           object
Price            float64
WalkScore        float64
Bike_Score       float64
Beds             float64
Bath             float64
Sqft             float64
Property_Type     object
Year_Built       float64
Status            object
dtype: object

In [21]:
#Normalization

df['Street']=df['Street'].apply(lambda x:x[0]+x[1:].lower())
df['City']=df['City'].apply(lambda x:x[0]+x[1:].lower())
df['State']=df['State'].apply(lambda x:x[0]+x[1:].lower())
df['Property_Type']=df['Property_Type'].apply(lambda x:x[0]+x[1:].lower())
df['Status']=df['Status'].apply(lambda x:x[0]+x[1:].lower())
df

,Street,City,County,State,Zipcode,Price,WalkScore,Bike_Score,Beds,Bath,Sqft,Property_Type,Year_Built,Status
0,1830 fillmore st #311,San francisco,San-Francisco,Ca,94115,1098000.0,0.0,0.0,2.0,2.0,1107.0,Condo,1984.0,Active
1,2152 16th ave,San francisco,San-Francisco,Ca,94116,1298000.0,0.0,0.0,3.0,2.0,1325.0,Single family residential,1938.0,Coming soon
2,2170 jackson st #1,San francisco,San-Francisco,Ca,94115,4525000.0,97.0,64.0,3.0,3.5,3428.0,Co-op,1925.0,Active
3,1631 la salle ave,San francisco,San-Francisco,Ca,94124,1495000.0,0.0,0.0,4.0,4.0,2430.0,Single family residential,1940.0,Active
4,1824 jackson st,San francisco,San-Francisco,Ca,94109,2300000.0,0.0,0.0,3.0,2.5,2854.0,Condo,1917.0,Active
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,36739 hafner st,Newark,Alameda-County,Ca,94560,1190000.0,80.0,50.0,4.0,2.0,1216.0,Single family residential,1954.0,Active
656,2214 browning st,Berkeley,Alameda-County,Ca,94702,1249000.0,0.0,0.0,3.0,2.0,1560.0,Single family residential,1918.0,Active
657,1510 e st #22,Hayward,Alameda-County,Ca,94541,498000.0,54.0,26.0,2.0,2.0,1120.0,Condo,1974.0,Active
658,1625 san lorenzo ave,Berkeley,Alameda-County,Ca,94707,1199000.0,93.0,78.0,2.0,2.0,1153.0,Single family residential,1925.0,Active


In [22]:
# One final check to remove duplicates
df.duplicated().value_counts()
df=df.drop_duplicates()
df

,Street,City,County,State,Zipcode,Price,WalkScore,Bike_Score,Beds,Bath,Sqft,Property_Type,Year_Built,Status
0,1830 fillmore st #311,San francisco,San-Francisco,Ca,94115,1098000.0,0.0,0.0,2.0,2.0,1107.0,Condo,1984.0,Active
1,2152 16th ave,San francisco,San-Francisco,Ca,94116,1298000.0,0.0,0.0,3.0,2.0,1325.0,Single family residential,1938.0,Coming soon
2,2170 jackson st #1,San francisco,San-Francisco,Ca,94115,4525000.0,97.0,64.0,3.0,3.5,3428.0,Co-op,1925.0,Active
3,1631 la salle ave,San francisco,San-Francisco,Ca,94124,1495000.0,0.0,0.0,4.0,4.0,2430.0,Single family residential,1940.0,Active
4,1824 jackson st,San francisco,San-Francisco,Ca,94109,2300000.0,0.0,0.0,3.0,2.5,2854.0,Condo,1917.0,Active
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,36739 hafner st,Newark,Alameda-County,Ca,94560,1190000.0,80.0,50.0,4.0,2.0,1216.0,Single family residential,1954.0,Active
656,2214 browning st,Berkeley,Alameda-County,Ca,94702,1249000.0,0.0,0.0,3.0,2.0,1560.0,Single family residential,1918.0,Active
657,1510 e st #22,Hayward,Alameda-County,Ca,94541,498000.0,54.0,26.0,2.0,2.0,1120.0,Condo,1974.0,Active
658,1625 san lorenzo ave,Berkeley,Alameda-County,Ca,94707,1199000.0,93.0,78.0,2.0,2.0,1153.0,Single family residential,1925.0,Active


In [23]:
df.kurt(numeric_only=True)

Price         231.909882
WalkScore      -1.696920
Bike_Score     -1.529852
Beds            4.455479
Bath           14.908133
Sqft           63.726760
Year_Built     -1.174032
dtype: float64

In [24]:
df['Price'].describe()

count    6.530000e+02
mean     1.568464e+06
std      2.229105e+06
min      9.000000e+04
25%      8.590000e+05
50%      1.198000e+06
75%      1.725000e+06
max      4.500000e+07
Name: Price, dtype: float64

In [25]:
df['Beds'].describe()

count    653.000000
mean       3.015314
std        1.428161
min        0.000000
25%        2.000000
50%        3.000000
75%        4.000000
max       12.000000
Name: Beds, dtype: float64

In [26]:
df['Bath'].describe()

count    653.000000
mean       2.186064
std        1.160537
min        0.000000
25%        1.500000
50%        2.000000
75%        2.500000
max       13.000000
Name: Bath, dtype: float64

In [27]:
df['Sqft'].describe()

count      653.000000
mean      1826.315467
std       1292.568410
min        375.000000
25%       1125.000000
50%       1483.000000
75%       2133.000000
max      20000.000000
Name: Sqft, dtype: float64

In [28]:
# Calculating Quartiles, IQR and removing the outliers 

In [29]:
Q1_Price = df['Price'].quantile(0.25)
Q3_Price = df['Price'].quantile(0.75)
IQR_Price = Q3_Price - Q1_Price
IQR_Price

866000.0

In [30]:
Q1_Beds = df['Beds'].quantile(0.25)
Q3_Beds = df['Beds'].quantile(0.75)
IQR_Beds = Q3_Beds - Q1_Beds
IQR_Beds

2.0

In [31]:
Q1_Bath = df['Bath'].quantile(0.25)
Q3_Bath = df['Bath'].quantile(0.75)
IQR_Bath = Q3_Bath - Q1_Bath
IQR_Bath

1.0

In [32]:
Q1_Sqft = df['Sqft'].quantile(0.25)
Q3_Sqft = df['Sqft'].quantile(0.75)
IQR_Sqft = Q3_Sqft - Q1_Sqft
IQR_Sqft

1008.0

In [33]:
df['Price'] = df['Price'][~((df['Price'] < (Q1_Price - 1.5 * IQR_Price)) |(df['Price'] > 
                                                                           (Q3_Price + 1.5 * IQR_Price)))]
df['Beds'] = df['Beds'][~((df['Beds'] < (Q1_Beds - 1.5 * IQR_Beds)) |(df['Beds'] > 
                                                                           (Q3_Beds + 1.5 * IQR_Beds)))]
df['Bath'] = df['Bath'][~((df['Bath'] < (Q1_Bath - 1.5 * IQR_Bath)) |(df['Bath'] > 
                                                                           (Q3_Bath + 1.5 * IQR_Bath)))]
df['Sqft'] = df['Sqft'][~((df['Sqft'] < (Q1_Sqft - 1.5 * IQR_Sqft)) |(df['Sqft'] > 
                                                                           (Q3_Sqft + 1.5 * IQR_Sqft)))]

/var/folders/2f/l2xlm54j57qdd4g5l_gz2jl00000gn/T/ipykernel_3807/2212149449.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Price'] = df['Price'][~((df['Price'] < (Q1_Price - 1.5 * IQR_Price)) |(df['Price'] >
/var/folders/2f/l2xlm54j57qdd4g5l_gz2jl00000gn/T/ipykernel_3807/2212149449.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Beds'] = df['Beds'][~((df['Beds'] < (Q1_Beds - 1.5 * IQR_Beds)) |(df['Beds'] >
/var/folders/2f/l2xlm54j57qdd4g5l_gz2jl00000gn/T/ipykernel_3807/2212149449.py:5: Setti

In [34]:
df.isna().sum() # checkinh number of outliers removed

Street            0
City              0
County            0
State             0
Zipcode           0
Price            34
WalkScore         0
Bike_Score        0
Beds             10
Bath             28
Sqft             37
Property_Type     0
Year_Built       19
Status            0
dtype: int64

In [35]:
df=df.dropna() #removing the null values. The outliers were removed and replaced by null values

In [36]:
df

,Street,City,County,State,Zipcode,Price,WalkScore,Bike_Score,Beds,Bath,Sqft,Property_Type,Year_Built,Status
0,1830 fillmore st #311,San francisco,San-Francisco,Ca,94115,1098000.0,0.0,0.0,2.0,2.0,1107.0,Condo,1984.0,Active
1,2152 16th ave,San francisco,San-Francisco,Ca,94116,1298000.0,0.0,0.0,3.0,2.0,1325.0,Single family residential,1938.0,Coming soon
3,1631 la salle ave,San francisco,San-Francisco,Ca,94124,1495000.0,0.0,0.0,4.0,4.0,2430.0,Single family residential,1940.0,Active
4,1824 jackson st,San francisco,San-Francisco,Ca,94109,2300000.0,0.0,0.0,3.0,2.5,2854.0,Condo,1917.0,Active
5,301 crescent ct #3101,San francisco,San-Francisco,Ca,94134,529000.0,17.0,42.0,1.0,1.0,804.0,Condo,2007.0,Active
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,36739 hafner st,Newark,Alameda-County,Ca,94560,1190000.0,80.0,50.0,4.0,2.0,1216.0,Single family residential,1954.0,Active
656,2214 browning st,Berkeley,Alameda-County,Ca,94702,1249000.0,0.0,0.0,3.0,2.0,1560.0,Single family residential,1918.0,Active
657,1510 e st #22,Hayward,Alameda-County,Ca,94541,498000.0,54.0,26.0,2.0,2.0,1120.0,Condo,1974.0,Active
658,1625 san lorenzo ave,Berkeley,Alameda-County,Ca,94707,1199000.0,93.0,78.0,2.0,2.0,1153.0,Single family residential,1925.0,Active


In [37]:
#Exporting to CSV

In [40]:
df.to_csv('scrapped_data',index=False)

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          bv                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 